In [ ]:
%reload_ext lab_black
import sqlite3
import pandas as pd

In [ ]:
batch_name = "boo_batch"
con = sqlite3.connect(
    f"/home/jupyter/tf/models/batch_run/{batch_name}/batch_results.sqlite"
)
cur = con.cursor()

In [ ]:
df = pd.read_csv(
    "/home/jupyter/tf/models/high_stress_OP_r0028/result_strain_item.csv", index_col=0
)

In [ ]:
df.reset_index()

In [ ]:
df.columns

In [ ]:
df.to_sql("testing", con, if_exists="append")

In [ ]:
sql_for_extract_all = 'SELECT * FROM testing'


In [ ]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

In [ ]:
print(cur.fetchall())

In [ ]:
df = pd.read_sql_query("SELECT * from taraban", con)

In [ ]:
df.testset.unique()